## Overview of GEE web interface

Although we will use the python API of GEE, it is good to take a look to the GEE Javascript playground.
We ca distinguish 4 main areas as the image below shows. 
1. Left panel. Here we have three tabs: <i>scripts</i>, <i>asset</i> and <i>docs</i>. 
2. Middle panel. A script editor.
3. Right panel. It consist of three tabs: <i>Inspector</i>, <i>console</i> and <i>tasks</i>. 
4. Map area. We can add visualize data it in this section or edit features.<br> 
![alt text](/files/images/GEE_overview.png "Create a folder in GEE")

### Data Visualization

GEE has a large amount of Earth data free available for researchers. However, sometimes we need to process our own data.

To add our images in GEE. We will create a new folder <b>Tutorial<b>  
![alt text](/files/images/create_folder_gee2.png "Create a folder in GEE")

We will load the image <i>subset.tif </i>. This is a subset of a Worldview-2 image and it was acquired in 2014.
Load the image in your GEE asset as we see in the image below. For now, keep the default options.
![alt text](/files/images/insert_image.png "Insert image in the tutorial folder")


Now that we can visualize the image loaded.
To do this, we will execute the commands below

In [51]:
import ee 
ee.Initialize() 
from IPython.display import Image  # To display image thumbnails.

In [64]:
# load our image 
image = ee.Image("users/rosamaguilar/tutorial/subset")
# display the image, The maximun pixel value is 2048 because the image has 11-bit per pixel but we are 
# making an 8-bit display image, then the pixel values need to be stretch to fit in the range (0,255).   
Image(url=image.getThumbUrl({'min':0, 'max':2048}))

Above we have an image but there is no clue about which bands are displayed and their order. 
We can specify bands to display and additional parameters as follows: <br><br>
<i>Bands:</i> 	List of three band names to be display in the RGB channels. <br> 
<i>palette:</i>	Color palette in a list of hexadecimal code strings. Apply to single-band images.<br> 
<i>opacity: </i>	The transparency of the layer (0.0 is fully transparent and 1.0 is fully opaque)<br> 
<i>format: </i>	 String to  specify either "jpg" or "png". <br>  

The following parameters can be fixed using a single number to be applied to the three bands or by using a list of three numbers, one for each band: <br> 
<i> min: </i>	Value(s) to map to 0. <br> 
<i> max: </i>	Value(s) to map to 255. <br> 
<i> gain: </i>	Value(s) by which to multiply each pixel value.<br> 
<i> bias: </i>	Value(s) to add to each pixel value.<br> 
<i> gamma: </i>	Gamma correction factor(s).<br> 


In [53]:
# set band order RGB natural color composite
# gamma corrections factors (one per band)
Image(url=image.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5,b3,b2', 'gamma': '0.95, 1.1, 1'}))

You may try other band combinations and gamma factors.

In [58]:
#We can also get image metadata information. 
print(image.getInfo())

{'type': 'Image', 'properties': {'system:asset_size': 5839050, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-5.2025191570512765, 12.159825582803961], [-5.202516851635505, 12.159825459718073], [-5.189927460512273, 12.15992551735162], [-5.189919034443333, 12.159932901288647], [-5.189909729639849, 12.159939196772044], [-5.189909178271649, 12.159942159884043], [-5.189992426450091, 12.170105809735732], [-5.189999980265262, 12.17011404853716], [-5.190006422302447, 12.170123145052345], [-5.190008690214821, 12.17012354831131], [-5.190010995831953, 12.170123671499278], [-5.202600866228118, 12.170023527763393], [-5.202609292178024, 12.170016143418682], [-5.202618596995535, 12.170009847540083], [-5.20261914824336, 12.170006884346753], [-5.2025354210176005, 12.159843320651753], [-5.202527867137842, 12.159835082220859], [-5.202521425008991, 12.159825985989212], [-5.2025191570512765, 12.159825582803961]]}}, 'version': 1490181720284000, 'id': 'users/rosamaguilar/tutorial/subset', 'band

Assuming we have a kml file with our vector data, namely parcels. We need to create a fusion table (ft) with that file. Afterward, we can use the id of the ft to create a GEE feature collection.

In [55]:
# create the feature collection from the fusion table
fc = ee.FeatureCollection ('ft:1K41m-umQ1K8Ys-9bVTqoEYzwKD6xEzIVTPELdVkK')

In [56]:
# listing the fields in the feature collection
print(fc.first().propertyNames().getInfo())

['name', 'description', 'id', 'class', 'system:index']


In [ ]:
# getting information about the Feature collection
print(fc.getInfo())

In [ ]:
# getting information about one feature (the first)
print(fc.first().getInfo())

In [ ]:
# counting the number of features in the collection
print(ee.Number(fc.aggregate_count('.all')).getInfo())

### Visualization using Folium

In a previous step we upload an image to a GEE asset and a vector layer to a google drive as a feature table. 
We will visualize them using folium. For more information about folium, visit: https://github.com/python-visualization/folium


In [13]:
# import GEE and foliumn
import ee
import folium
ee.Initialize()

In [4]:
# define a couple of function to interact with GEE
# from https://github.com/python-visualization/folium_contrib/blob/master/GoogleEarthEngine_layer.ipynb
def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """ Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    print(folium_kwargs['overlay'])
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)

In [5]:
# lets draw our fc
fc = ee.FeatureCollection ('ft:1K41m-umQ1K8Ys-9bVTqoEYzwKD6xEzIVTPELdVkK')
geojs = fc.getDownloadURL("json")

In [6]:
print(geojs)

https://earthengine.googleapis.com/api/table?docid=a86fd24bec054241ef01e03d3f45c8e8&token=d57c479d08c2a72621b42e5b082414fe


In [33]:
# lets retrieve the fc as a json to display in our map
import urllib
with urllib.request.urlopen(geojs) as url:
    fcjs = url.read()
fcjs = str(fcjs, 'utf-8')  
#print(fcjs)

{"type":"FeatureCollection","features":[{"geometry":{"coordinates":[[[-5.199613999999999,12.165225000000001],[-5.1990310000000015,12.165136],[-5.198975,12.165533],[-5.199543,12.165571],[-5.199613999999999,12.165225000000001]]],"geodesic":true,"type":"Polygon"},"id":"2","properties":{"class":5.0,"description":"","id":1.0,"name":""},"type":"Feature"},{"geometry":{"coordinates":[[[-5.201773,12.16407],[-5.200976999999998,12.164202999999999],[-5.201564,12.164885],[-5.201773,12.16407]]],"geodesic":true,"type":"Polygon"},"id":"3","properties":{"class":2.0,"description":"","id":2.0,"name":""},"type":"Feature"},{"geometry":{"coordinates":[[[-5.1967729999999985,12.167687000000004],[-5.196262,12.167743999999997],[-5.196262,12.168028],[-5.196488999999999,12.168256],[-5.1967729999999985,12.168198999999996],[-5.1967729999999985,12.167687000000004]]],"geodesic":true,"type":"Polygon"},"id":"4","properties":{"class":4.0,"description":"","id":4.0,"name":""},"type":"Feature"},{"geometry":{"coordinates":[

In [50]:
# add our image "subset"
m1 = folium.Map(location=[12.1724,-5.1874], zoom_start= 14)
image = ee.Image("users/rosamaguilar/tutorial/subset")  
vis_params = {'min': 0, 'max':2048, 'bands':'b5,b3,b2'}
m1.choropleth(geo_str=fcjs,fill_opacity=0.7, fill_color='red') 
folium_gee_layer(m1,image,vis_params,folium_kwargs={'overlay':True,'name':'Subset'})
m1.add_child(folium.LayerControl())
m1

True
